<a href="https://colab.research.google.com/github/senecamiller/Coding_02_Final_Project/blob/main/Visualization_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving final_df_cleaned.csv to final_df_cleaned.csv


In [2]:
import pandas as pd

# Load the uploaded file
final_df_cleaned = pd.read_csv('final_df_cleaned.csv')
print(final_df_cleaned.head())

      price  location  year    ttaf serial number            registration  \
0       NaN    Poland  2021    490h       24-0345    SP-WKX (EASA-Poland)   
1  460000.0    Poland  1974  15877h     172S10770    SP-THL (EASA-Poland)   
2       NaN    Poland  1978   4654h     F15001127    SP-KIZ (EASA-Poland)   
3       NaN  Portugal  1959   2830h     T20608855  SP-HAV (EASA - Poland)   
4  419000.0    Poland  1991   3392h          3630  SP-SZP (EASA - Poland)   

                       reason of sale availability  type of flights  \
0  Other aviation project in progress  Immediately  Private Flights   
1  Other aviation project in progress  Immediately  Private Flights   
2  Other aviation project in progress  Immediately  Flight training   
3  Other aviation project in progress  Immediately              NaN   
4  Other aviation project in progress  Immediately  Private flights   

                    arc  ...      documents since new  \
0             Valid ARC  ...                      NaN

In [3]:
import plotly
import plotly.express as px
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

--- Remaining Engine Hours by Plane Location ---

In [4]:
# Make a copy of ['remaining hours on engine']  and ['location'] columns
rhoe_cleaned = final_df_cleaned['remaining hours on engine'].copy()
location_copy = final_df_cleaned['location'].copy()

# Filter unwanted values and NaNs
unwanted_values = ['On condition', '396h+ 200h extension']
filtered_df = final_df_cleaned[
    ~final_df_cleaned['remaining hours on engine'].isin(unwanted_values)
    ].dropna(subset=['remaining hours on engine', 'location'])

# Remove 'h' from the 'remaining hours on engine' column
filtered_df['remaining hours on engine'] = (
    filtered_df['remaining hours on engine']
    .str.replace(r'h', '', regex=True)
    .pipe(pd.to_numeric, errors='coerce')
    )

# Group by ['location'], find max/min for ['remaining hours on engine']
stats_by_location = filtered_df.groupby('location')['remaining hours on engine'].agg(
    max_hours='max',
    min_hours='min'
).reset_index()

# Sort the resulting dataframe by 'max_hours' in descending order
stats_by_location = stats_by_location.sort_values(by='max_hours', ascending=False)

stats_by_location

,location,max_hours,min_hours
2,Portugal,3450,5
5,Spain,2692,912
1,Poland,2400,38
0,Czech Republic,1950,1736
4,Slovenia,1194,488
3,Romania,948,948


In [5]:
# Remove the ' h' addition to numerical columns for proper plotting
stats_by_location['max_hours'] = stats_by_location['max_hours'].astype(float)
stats_by_location['min_hours'] = stats_by_location['min_hours'].astype(float)

# Melt the DataFrame for grouped bar plotting
plot_df = stats_by_location.melt(id_vars='location', var_name='Hour Type', value_name='Hours')

# Create chart
title = "Maximum and Minimum Remaining Hours on Engine Based on Plane Location"
chart_size = {'height': 550, 'width': 1200}

fig = px.bar(
    plot_df,
    x='location',
    y='Hours',
    color='Hour Type',
    color_discrete_map={'max_hours':'thistle', 'min_hours':'skyblue'},
    title=title,
    template='ggplot2',
    text='Hours'
)

# Update layout
fig.update_layout(
    chart_size,
    xaxis_title="",
    yaxis_title="",
    title_x=0.5,
    font_size=15,
    legend_title=None,
    legend=dict(x=0.85, y=0.8),
    plot_bgcolor='white',
    title_subtitle_text="'On condition' and NaN values were removed. The data was grouped by plane sale location.",
    barmode='group'
)

fig.update_xaxes(ticks = "")

fig.update_yaxes(showticklabels=False,
                 ticks = "")

# Format text labels to include ' h'
fig.update_traces(texttemplate='%{text:.0f} h',
                  textposition='inside'
                  )

# Show the chart
fig.show()